In [81]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [82]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [83]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [84]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '###' #'exact'
configuration.host = "https://exact.cs.fau.de" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download Rect annotation

In [85]:
annotation_types_points = {}
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
            
for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.name] = annotation_type

annotation_types_points

{'L0': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 384,
  'name': 'L0',
  'node_count': 0,
  'product': 104,
  'sort_order': 0,
  'vector_type': 1},
 'L1': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 385,
  'name': 'L1',
  'node_count': 0,
  'product': 104,
  'sort_order': 1,
  'vector_type': 1},
 'L2': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 386,
  'name': 'L2',
  'node_count': 0,
  'product': 104,
  'sort_order': 2,
  'vector_type': 1},
 'L3': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': Fa

In [86]:
annotation_types_ref_rect

{'area_hit_test': True,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 5000,
 'default_width': 5000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 412,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 105,
 'sort_order': 0,
 'vector_type': 1}

In [87]:
annotations = []

image_set = image_sets_api.list_image_sets(name="Cyto_Aperio", expand="product_set,product_set.annotationtype_set").results[0]


for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id, annotation_type=annotation_types_ref_rect.id, # BoundBox ID
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        annotations.append([image.id, image.name, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "id", "vector", "unique_identifier", "annotation_type"])
annotations.head()

,image_id,image_name,id,vector,unique_identifier,annotation_type
0,10392,A_BB_574162_1.svs,2754771,"{'x1': 2104, 'x2': 28849, 'y1': 2210, 'y2': 28...",5f2be120-fc2e-4c72-afa4-a3d4ba9c86e7,412
1,10389,A_BB_568381_1.svs,2754772,"{'x1': 2634, 'x2': 30395, 'y1': 2927, 'y2': 28...",b82de944-fca6-4708-bdd8-5d8714daceb7,412
2,10387,A_BB_568320_1.svs,2754773,"{'x1': 2362, 'x2': 29798, 'y1': 3279, 'y2': 29...",f0289b0a-a700-4bcb-aa78-f2e7097819b9,412
3,10384,A_BB_563479_1.svs,2754774,"{'x1': 6186, 'x2': 32708, 'y1': 4456, 'y2': 32...",4de1932e-0b82-403e-8173-eccefade512b,412
4,10378,A_BB_563476_1.svs,2754775,"{'x1': 2336, 'x2': 29860, 'y1': 2826, 'y2': 30...",fa22bbe8-0d8c-415d-8d42-4f014054f582,412


## Load GT and find closest point

In [88]:
Path("../../../../SDATA/Statistics/SDATA_Final_Annotations.pkl").exists()

True

In [89]:
df = pd.read_pickle("../../../../SDATA/Statistics/SDATA_Final_Annotations.pkl")

df["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in df["vector"]]

df["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in df["vector"]]

df["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(df["x1"], df["x2"])]
df["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(df["y1"], df["y2"])]

df["anno_width"] = [x2-x1 for x1, x2 in zip(df["x1"], df["x2"])]
df["anno_height"]= [y2-y1 for y1, y2 in zip(df["y1"], df["y2"])]

df = df.sort_values(by=["grade"], ascending=False)
df.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,user_id,deleted,...,data_set_name,version,x1,y1,x2,y2,center_x,center_y,anno_width,anno_height
unique_identifier,,,,,,,,,,,,,,,,,,,,,
c0eac559-6184-4500-b91f-556b1bc3b19d,2460370,5166,240,equine,20_EIPH_576150 berliner blau.svs,Prussian,4,"{'x1': 22046, 'x2': 22144, 'y1': 27764, 'y2': ...",1,False,...,SREP,Density,22046.0,27764.0,22144.0,27862.0,22095.0,27813.0,98.0,98.0
0e97284b-9764-43e8-b077-60d5ab366db2,2344354,5200,269,equine,14_EIPH_568381 Turnbull blue.svs,Turnbull,4,"{'x1': 24442, 'x2': 24528, 'y1': 22640, 'y2': ...",1,False,...,SREP,Density,24442.0,22640.0,24528.0,22726.0,24485.0,22683.0,86.0,86.0
76bad88e-5708-475b-9de4-8ba6d8e9169d,2344374,5200,269,equine,14_EIPH_568381 Turnbull blue.svs,Turnbull,4,"{'x1': 28238, 'x2': 28340, 'y1': 15269, 'y2': ...",1,False,...,SREP,Density,28238.0,15269.0,28340.0,15371.0,28289.0,15320.0,102.0,102.0
7c86d6d4-9764-468e-86ee-aa6b221e75a2,2344371,5200,269,equine,14_EIPH_568381 Turnbull blue.svs,Turnbull,4,"{'x1': 26802, 'x2': 26902, 'y1': 13212, 'y2': ...",1,False,...,SREP,Density,26802.0,13212.0,26902.0,13312.0,26852.0,13262.0,100.0,100.0
9641c6f1-d045-4a0a-b337-10ea8ee22fec,2344368,5200,269,equine,14_EIPH_568381 Turnbull blue.svs,Turnbull,4,"{'x1': 10663, 'x2': 10745, 'y1': 6630, 'y2': 6...",1,False,...,SREP,Density,10663.0,6630.0,10745.0,6712.0,10704.0,6671.0,82.0,82.0


In [90]:
num_lines = 5
patch_size = 50
search_point_radius = 0.15

registration_points = []
for id, bounding_box in annotations.iterrows():
    
    df_gt = df[df["image_name"].str.contains(bounding_box.image_name.split("_")[-2])]
    
    box_width = bounding_box.vector["x2"] - bounding_box.vector["x1"]
    box_height = bounding_box.vector["y2"] - bounding_box.vector["y1"]
    
    step_size_x = box_width // num_lines
    step_size_y = box_height // num_lines
    
    gt_search_point_radius = int(step_size_x * search_point_radius)
    
    point_index = 0
    for x in range(bounding_box.vector["x1"] + step_size_x //2, bounding_box.vector["x2"], step_size_x):
        for y in range(bounding_box.vector["y1"] + step_size_y //2, bounding_box.vector["y2"], step_size_y):
            
            ref_point = annotation_types_points[f"L{point_index}"]
            
            # find closest annotation in range
            temp = df_gt[(df_gt["center_x"] > x - gt_search_point_radius) & 
                          (df_gt["center_x"] < x + gt_search_point_radius) & 
                          (df_gt["center_y"] > y - gt_search_point_radius) & 
                          (df_gt["center_y"] < y + gt_search_point_radius)].copy()
            
            temp["distance"] = [abs(cx - x) + abs(cy - y) for cx, cy in zip(temp["center_x"], temp["center_y"])]
            temp = temp.sort_values(by=["grade", "distance"], ascending=[False, True])
        
            point = None
            for uuid, gt_anno in temp.iterrows():
                
                vector = {"x1": gt_anno.vector["x1"], "y1": gt_anno.vector["y1"], 
                          "x2": gt_anno.vector["x2"], "y2": gt_anno.vector["y2"]}
                
                
                point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id, uuid]
                break
            
            if point is None: # No ref point found
                
                vector = {"x1": x - patch_size, "y1": y - patch_size, 
                          "x2": x + patch_size, "y2": y + patch_size}
                
                point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id, None]
                
            registration_points.append(point)
                
            point_index += 1
    
registration_points = pd.DataFrame(registration_points, 
                                   columns=["image_id", "vector", "point_index", "point_name", "point_id", "gt_uuid"])
registration_points

,image_id,vector,point_index,point_name,point_id,gt_uuid
0,10392,"{'x1': 4728, 'y1': 4826, 'x2': 4828, 'y2': 4926}",0,L0,384,None
1,10392,"{'x1': 4671, 'y1': 10586, 'x2': 4759, 'y2': 10...",1,L1,385,72f12897-7d3c-4458-a7d8-7fdc8e7176ab
2,10392,"{'x1': 5100, 'y1': 15401, 'x2': 5208, 'y2': 15...",2,L2,386,a9d37e23-8267-49b1-a637-6c889bd79afa
3,10392,"{'x1': 5230, 'y1': 21108, 'x2': 5336, 'y2': 21...",3,L3,387,42096c63-212d-4a51-bf00-e8d34de09f47
4,10392,"{'x1': 5387, 'y1': 25652, 'x2': 5497, 'y2': 25...",4,L4,388,56efe99e-f891-4ad4-a9ee-e096ac3916aa
...,...,...,...,...,...,...
120,10378,"{'x1': 27054, 'y1': 5581, 'x2': 27154, 'y2': 5...",20,L20,407,None
121,10378,"{'x1': 27491, 'y1': 11331, 'x2': 27612, 'y2': ...",21,L21,408,39f211cd-034b-4013-9271-439fef05afb9
122,10378,"{'x1': 26509, 'y1': 16374, 'x2': 26624, 'y2': ...",22,L22,409,142f2230-ec26-4609-a5a5-93387387453d
123,10378,"{'x1': 26949, 'y1': 22715, 'x2': 27048, 'y2': ...",23,L23,410,1e9cac70-7c73-4abb-8f8c-4170a0b1b35e


## Upload ref Points to EXACT

In [91]:
annotations_to_upload = []

for id, bounding_box in tqdm(registration_points.iterrows()):
    
    
    exact_annos =  annotations_api.list_annotations(image=bounding_box.image_id, annotation_type=bounding_box.point_id).results
    
    if len(exact_annos) == 0:
        anno = Annotation(annotation_type=bounding_box.point_id, 
                          vector=bounding_box.vector, 
                          image=bounding_box.image_id,
                         unique_identifier=bounding_box.gt_uuid)

        annotations_to_upload.append(anno)

    else:
        prinf(f"ImageId: {bounding_box.image_id}, AnnotationType: {bounding_box.point_id} combination allready exists!")

125it [00:24,  5.09it/s]


In [92]:
annotations_to_upload[:3]

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10392,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 4728, 'x2': 4828, 'y1': 4826, 'y2': 4926},
  'verified_by_user': None},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10392,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '72f12897-7d3c-4458-a7d8-7fdc8e7176ab',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 4671, 'x2': 4759, 'y1': 10586, 'y2': 10674},
  'verified_by_user': None},
 {'annotation_type': 386,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  

## Check that no error occurred and afterwards upload annotations

In [94]:
#annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2754781,
  'image': 10392,
  'last_edit_time': datetime.datetime(2021, 1, 10, 11, 58, 15, 322322),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 11, 58, 15, 322297),
  'unique_identifier': 'e4d98794-d4fb-404b-a4ae-4b312b468e8e',
  'uploaded_media_files': [],
  'user': 1,
  'vector': {'x1': 4728, 'x2': 4828, 'y1': 4826, 'y2': 4926},
  'verified_by_user': 'False'},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2754782,
  'image': 10392,
  'last_edit_time': datetime.datetime(2021, 1, 10, 11, 58, 15, 322389),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 11, 58, 15, 322375),
  'unique_identifier': '72f12897-7d3c-4458-a7d8-7fdc8e7176ab',
  'uploaded_media_files': [],